## Търсене и извличане на информация. Приложение на дълбоко машинно обучение
> ### Стоян Михов
> #### Зимен семестър 2021/2022

### Упражнение 8

 За да работи програмата трябва корпуса от публицистични текстове за Югоизточна Европа,
 да се намира разархивиран в директорията, в която е програмата (виж упражнение 2).

 Преди да се стартира програмата е необходимо да се активира съответното обкръжение с командата: `conda activate tii`
 
 Настоящата програма използва библиотеката sklearn.
 За да я инсталирате, след активиране на обкръжението трябва да изпълните командата: `conda install scikit-learn`

In [ ]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import sys
import random
import math
import pprint

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import numpy as np
from sklearn.decomposition import TruncatedSVD

######  Визуализация на прогреса

In [ ]:
class progressBar:
    def __init__(self ,barWidth = 50):
        self.barWidth = barWidth
        self.period = None
    def start(self, count):
        self.item=0
        self.period = int(count / self.barWidth)
        sys.stdout.write("["+(" " * self.barWidth)+"]")
        sys.stdout.flush()
        sys.stdout.write("\b" * (self.barWidth+1))
    def tick(self):
        if self.item>0 and self.item % self.period == 0:
            sys.stdout.write("-")
            sys.stdout.flush()
        self.item += 1
    def stop(self):
        sys.stdout.write("]\n")

######  Разбиване на корпус на тестов и тренировъчен

In [ ]:
def splitSentCorpus(fullSentCorpus, testFraction = 0.1):
    random.seed(42)
    random.shuffle(fullSentCorpus)
    testCount = int(len(fullSentCorpus) * testFraction)
    testSentCorpus = fullSentCorpus[:testCount]
    trainSentCorpus = fullSentCorpus[testCount:]
    return testSentCorpus, trainSentCorpus

####   Мултиномен Бейсов класификатор от упражнение 4

In [ ]:
def trainMultinomialNB(trainClassCorpus):
    N = sum(len(classList) for classList in trainClassCorpus)
    classesCount = len(trainClassCorpus)
    pb = progressBar(50)
    pb.start(N)
    V = {}
    for c in range(classesCount):
        for text in trainClassCorpus[c]:
            pb.tick()
            terms = [ token.lower() for token in text if token.isalpha() ]
            for term in terms:
                if term not in V:
                    V[term] = [0] * classesCount
                V[term][c] += 1
    pb.stop()

    Nc = [ (len(classList)) for classList in trainClassCorpus ]
    prior = [ Nc[c] / N for c in range(classesCount) ]
    T = [0] * classesCount
    for t in V:
        for c in range(classesCount):
            T[c] += V[t][c]
    condProb = {}
    for t in V:
        condProb[t] = [ (V[t][c] +1) / (T[c] + len(V)) for c in range(classesCount)]
    return condProb, prior, V

In [ ]:
def applyMultinomialNB(prior, condProb, text, features = None ):
    terms = [ token.lower() for token in text if token.isalpha() ]
    for c in range(len(prior)):
        score = math.log(prior[c])
        for t in terms:
            if t not in condProb: continue
            if features and t not in features: continue
            score += math.log(condProb[t][c])
        if c == 0 or score > maxScore:
            maxScore = score
            answer = c
    return answer

In [ ]:
def testClassifier(testClassCorpus, gamma):
    L = [ len(c) for c in testClassCorpus ]
    pb = progressBar(50)
    pb.start(sum(L))
    classesCount = len(testClassCorpus)
    confusionMatrix = [ [0] * classesCount for _ in range(classesCount) ]
    for c in range(classesCount):
        for text in testClassCorpus[c]:
            pb.tick()
            c_MAP = gamma(text)
            confusionMatrix[c][c_MAP] += 1
    pb.stop()
    precision = []
    recall = []
    Fscore = []
    for c in range(classesCount):
        extracted = sum(confusionMatrix[x][c] for x in range(classesCount))
        if confusionMatrix[c][c] == 0:
            precision.append(0.0)
            recall.append(0.0)
            Fscore.append(0.0)
        else:
            precision.append( confusionMatrix[c][c] / extracted )
            recall.append( confusionMatrix[c][c] / L[c] )
            Fscore.append((2.0 * precision[c] * recall[c]) / (precision[c] + recall[c]))
    P = sum( L[c] * precision[c] / sum(L) for c in range(classesCount) )
    R = sum( L[c] * recall[c] / sum(L) for c in range(classesCount) )
    F1 = (2*P*R) / (P + R)
    print('=================================================================')
    print('Матрица на обърквания: ')
    for row in confusionMatrix:
        for val in row:
            print('{:4}'.format(val), end = '')
        print()
    print('Прецизност: '+str(precision))
    print('Обхват: '+str(recall))
    print('F-оценка: '+str(Fscore))
    print('Обща презизност: '+str(P))
    print('Общ обхват: '+str(R))
    print('Обща F-оценка: '+str(F1))
    print('=================================================================')
    print()

####   Влагане на думи в нискомерно гъсто векторно пространство от упражнение 6

In [ ]:
def extractDictionary(corpus, limit=20000):
    pb = progressBar()
    pb.start(len(corpus))
    dictionary = {}
    for doc in corpus:
        pb.tick()
        for w in doc:
            if w not in dictionary: dictionary[w] = 0
        dictionary[w] += 1
    L = sorted([(w,dictionary[w]) for w in dictionary], key = lambda x: x[1] , reverse=True)
    if limit > len(L): limit = len(L)
    words = [ w for w,_ in L[:limit] ]
    word2ind = { w:i for i,w in enumerate(words)}
    pb.stop()
    return words, word2ind

In [ ]:
def co_occurrence_matrix(corpus, window_size=4, limit=20000):
    words, word2ind = extractDictionary(corpus,limit=limit)
    num_words = len(words)
    X=np.zeros((num_words,num_words))
    pb = progressBar()
    pb.start(len(corpus))
    for doc in corpus:
        pb.tick()
        for wi in range(len(doc)):
            if doc[wi] not in word2ind: continue
            i=word2ind[doc[wi]]
            for k in range(1,window_size+1):
                if wi-k>=0 and doc[wi-k] in word2ind:
                    j=word2ind[doc[wi-k]]
                    X[i,j] += 1
                if wi+k<len(doc) and doc[wi+k] in word2ind:
                    j=word2ind[doc[wi+k]]
                    X[i,j] += 1
    pb.stop()
    return X, words, word2ind

In [ ]:
def PMI_matrix(C):
    rowSums = np.sum(C,axis=1)
    colSums = np.sum(C,axis=0)
    D = np.sum(rowSums)
    Z = np.outer(rowSums,colSums)
    X = np.maximum(np.log( D * C / Z),0)
    return X

In [ ]:
def SVD_k_dim(X, k=100, n_iters = 10):
    # Документация на метода има на https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
    
    print("Running Truncated SVD over %i words..." % (X.shape[0]))
    svd = TruncatedSVD(n_components=k, n_iter=n_iters)
    svd.fit(X)
    X_reduced = svd.transform(X)
    print("Done.")
    return X_reduced

######  Влагане на документи в нискомерно гъсто векторно пространство

In [ ]:
def docVector(document, Embedding, word2ind):
    tf = np.zeros(len(word2ind))
    for w in document:
        if w in word2ind:
            tf[word2ind[w]] += 1
    d=np.dot(tf,Embedding)
    return d / np.linalg.norm(d)

In [ ]:
def corpusEmbedding(corpus, Embedding, word2ind):
    return np.stack([ docVector(doc, Embedding, word2ind) for doc in corpus ])

####    Логистична регресия -- Бинарен класификатор

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def crossEntropyS(X, Y, w, b):
    ### X е с размерност: (m,n)
    ### Y е с размерност: (m,)
    ### W е с размерност: (n,)
    ### b е с размерност: (1,)
    s = sigmoid(np.dot(X,w)+b)
    pr = (1-Y) + (2*Y-1)*s
    ce = -np.mean(np.log(pr))
    return ce

In [ ]:
def gradCrossEntropyS(X,Y,w,b):
    g = Y - sigmoid(np.dot(X,w)+b)
    db = -np.mean(g)
    dw = -np.mean( g[:,np.newaxis] * X,axis=0)
    return dw, db

####    Логистична регресия -- класификатор при много класове

In [ ]:
def softmaxV(u):
    ### u следва да бъде вектор с резмер k
    e = np.exp(u)
    return e / np.sum(e)

In [ ]:
def softmaxM(U):
    ### U следва да бъде матрица с размерност: (m,k)
    e = np.exp(U)
    return e / np.sum(e,axis=1)[:,np.newaxis]

In [ ]:
def crossEntropyM(X, Y, W, b):
    ### класовете са k
    ### X е с размерност: (m,n)
    ### Y е с размерност: (m,)
    ### W е с размерност: (n,k)
    ### b е с размерност: (k,)

    s = softmaxM(np.dot(X,W)+b[np.newaxis,:]) ### (m,k)
    m = X.shape[0]
    pr = s[np.arange(m),Y]
    ce = -np.mean(np.log(pr))
    return ce

In [ ]:
def gradCrossEntropyM(X,Y,W,b):
    m = X.shape[0]
    k = W.shape[1]
    s = softmaxM(np.dot(X,W)+b[np.newaxis,:])
    v = -s
    v[np.arange(m),Y] += 1.
    db = -np.mean(v, axis=0)
    dW = -(1/m) * np.dot(X.transpose(), v)
    return dW, db

####    спускане по градиента

In [ ]:
def gradientDescend(X,Y,tX,tY,w0,b0,crossEntropy,gradCrossEntropy,alpha=1.):
    epoch=0
    w=w0
    b=b0
    ceList = []
    tceList = []
    while epoch<100000:
        if epoch % 1000 == 0:
            ce = crossEntropy(X, Y, w, b)
            tce = crossEntropy(tX, tY, w, b)
            print(epoch,ce,tce)
            ceList.append(ce)
            tceList.append(tce)
        epoch += 1
        dw, db = gradCrossEntropy(X,Y,w,b)
        b -= alpha * db
        w -= alpha * dw
    return w,b,ceList,tceList

######   Зареждане на корпуса

In [ ]:
corpus_root = 'JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
startToken = '<START>'
endToken = '<END>'
fileNames = myCorpus.fileids()

ecoCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('E-Economy'+'/')==0 ]
milCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('S-Military'+'/')==0 ]
polCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('J-Politics'+'/')==0 ]
culCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('C-Culture'+'/')==0 ]
socCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('D-Society'+'/')==0 ]
zCorpus = [ [startToken] + [w.lower() for w in myCorpus.words(f)] + [endToken] for f in fileNames if f.find('Z'+'/')==0 ]

testEcoCorpus, trainEcoCorpus = splitSentCorpus(ecoCorpus)
testMilCorpus, trainMilCorpus = splitSentCorpus(milCorpus)
testPolCorpus, trainPolCorpus = splitSentCorpus(polCorpus)
testCulCorpus, trainCulCorpus = splitSentCorpus(culCorpus)

######   Тестване на Бейсов класификатор

In [ ]:
condProbM, priorM, VM = trainMultinomialNB([trainEcoCorpus,trainMilCorpus,trainPolCorpus,trainCulCorpus])

In [ ]:
gamma = lambda text : applyMultinomialNB(priorM, condProbM, text)

In [ ]:
testClassifier([testEcoCorpus,testMilCorpus,testPolCorpus,testCulCorpus], gamma)

In [ ]:
condProbM, priorM, VM = trainMultinomialNB([trainEcoCorpus,trainMilCorpus])

In [ ]:
gamma = lambda text : applyMultinomialNB(priorM, condProbM, text)

In [ ]:
testClassifier([testEcoCorpus,testMilCorpus], gamma)

####  Влагане на думите

In [ ]:
C, words, word2ind = co_occurrence_matrix(ecoCorpus+milCorpus+polCorpus+culCorpus+socCorpus+zCorpus)
X = PMI_matrix(C)
X_reduced = SVD_k_dim(X)

X_lengths_100d = np.linalg.norm(X_reduced, axis=1)
X_normalized_100d = X_reduced / X_lengths_100d[:, np.newaxis]

###### бинарно класифициране

In [ ]:
trainX = np.concatenate((
                         corpusEmbedding(trainEcoCorpus,X_normalized_100d,word2ind),
                         corpusEmbedding(trainMilCorpus,X_normalized_100d,word2ind)
                         ))
trainY = np.concatenate((
                         np.ones(len(trainEcoCorpus),dtype='int32')*0,
                         np.ones(len(trainMilCorpus),dtype='int32')*1
                         ))
testX = np.concatenate((
                        corpusEmbedding(testEcoCorpus,X_normalized_100d,word2ind),
                        corpusEmbedding(testMilCorpus,X_normalized_100d,word2ind)
                        ))
testY = np.concatenate((
                        np.ones(len(testEcoCorpus),dtype='int32')*0,
                        np.ones(len(testMilCorpus),dtype='int32')*1
                        ))

w0 = np.random.normal(0.,1.,100)
b0 = np.random.normal(0., 1., 1)

In [ ]:
w,b,ceList,tceList = gradientDescend(trainX,trainY,testX,testY,w0,b0,crossEntropyS,gradCrossEntropyS,alpha=1.)

In [ ]:
%matplotlib widget

In [ ]:
plt.plot([*range(len(ceList))],ceList)
plt.plot([*range(len(tceList))],tceList)
plt.show()

In [ ]:
gamma = lambda text : 1 if sigmoid(np.dot(w,docVector(text, X_normalized_100d, word2ind))+b)>0.5 else 0

In [ ]:
testClassifier([testEcoCorpus,testMilCorpus], gamma)

###### много класове

In [ ]:
trainX = np.concatenate((
                         corpusEmbedding(trainEcoCorpus,X_normalized_100d,word2ind),
                         corpusEmbedding(trainMilCorpus,X_normalized_100d,word2ind),
                         corpusEmbedding(trainPolCorpus,X_normalized_100d,word2ind),
                         corpusEmbedding(trainCulCorpus,X_normalized_100d,word2ind)
                         ))
trainY = np.concatenate((
                         np.ones(len(trainEcoCorpus),dtype='int32')*0,
                         np.ones(len(trainMilCorpus),dtype='int32')*1,
                         np.ones(len(trainPolCorpus),dtype='int32')*2,
                         np.ones(len(trainCulCorpus),dtype='int32')*3
                         ))

testX = np.concatenate((
                         corpusEmbedding(testEcoCorpus,X_normalized_100d,word2ind),
                         corpusEmbedding(testMilCorpus,X_normalized_100d,word2ind),
                         corpusEmbedding(testPolCorpus,X_normalized_100d,word2ind),
                         corpusEmbedding(testCulCorpus,X_normalized_100d,word2ind)
                         ))
testY = np.concatenate((
                         np.ones(len(testEcoCorpus),dtype='int32')*0,
                         np.ones(len(testMilCorpus),dtype='int32')*1,
                         np.ones(len(testPolCorpus),dtype='int32')*2,
                         np.ones(len(testCulCorpus),dtype='int32')*3
                         ))

W0 = np.random.normal(0.,1.,size=(100,4))
b0 = np.random.normal(0., 1., 4)

In [ ]:
W,b,ceList,tceList = gradientDescend(trainX,trainY,testX,testY,W0,b0,crossEntropyM,gradCrossEntropyM,alpha=1.)

In [ ]:
# (W1,b1,ceList1,tceList1) = (np.array([[ 1.24766566e+01, -4.22387240e+00, -8.25850069e-01, -5.56209196e+00], [-2.14271994e+00, -1.56167498e+00, 1.96770505e+00, -9.28385583e-01], [ 2.64505538e-01, 6.57290091e-01, -7.36889722e+00, 7.45165459e+00], [-3.85095108e+01, 1.37912821e+01, 1.59747492e+01, 6.17305038e+00], [ 1.29104648e+01, -8.85889393e+00, -4.27818413e+00, -1.47056063e+00], [ 4.46298699e+00, -8.02124115e+00, -3.63188480e+00, 6.14496705e+00], [-4.22033025e+00, 7.21095791e+00, -1.59125117e+01, 9.26907765e+00], [ 6.42120461e+00, -4.02745894e+00, 3.66132823e+00, -1.74326765e+00], [ 9.30117595e+00, -1.21768992e+01, -7.69643891e+00, 1.37863711e+01], [ 7.24501860e+00, 3.94209669e+00, -4.17238791e+00, -1.03939233e+01], [-7.41920869e+00, -6.79342039e+00, 1.15444711e+01, 8.36818782e-01], [-4.36442389e+00, -4.80182666e+00, -1.40294142e+00, 7.08077743e+00], [ 4.62013710e+00, -1.35791815e+01, 1.11601912e+01, -5.75661762e+00], [ 4.83664616e+00, -9.18555702e+00, -5.25417454e+00, 1.15455582e+01], [ 9.94667258e+00, 1.06601428e+01, -8.63593268e+00, -1.18113208e+01], [ 1.63882334e+00, 2.02559232e+00, -9.44906680e+00, 3.49221107e+00], [-8.97231627e+00, -1.92153393e+00, 6.52467148e+00, 5.61513793e+00], [ 1.33785369e+01, 1.80748843e+00, -6.29871101e+00, -8.65913408e+00], [-2.96437643e+00, -3.05355435e+00, 7.22574218e+00, 4.78958204e-01], [-4.27854716e+00, -4.46771008e+00, 8.54500359e+00, -2.45449520e+00], [-1.32077613e+01, -5.83731669e-01, 1.37703624e+01, 3.03070465e+00], [-3.11725999e-01, 1.69411087e+01, -4.13127220e+00, -1.05225417e+01], [ 1.25729417e+00, -9.75414231e+00, 2.61648844e+00, 4.85135532e+00], [-6.90751869e+00, 4.33363985e+00, 5.53513497e-01, -5.03058872e-01], [ 5.70722531e+00, -3.51853337e+00, 1.36072578e+00, -4.07487291e+00], [ 2.52219505e+00, 3.48327611e+00, -8.62082111e+00, 4.22850927e+00], [ 4.45017733e+00, 6.80379100e+00, -1.09906705e+01, -1.27171855e+00], [ 7.49226298e-01, -1.02387301e+00, 2.55126021e+00, -3.10420958e+00], [ 6.46801886e+00, -6.59989453e+00, 4.44283855e+00, -4.06408022e+00], [-1.11078056e+00, 1.35552180e+00, -4.00277052e+00, 3.91355291e+00], [ 1.11538367e+01, -1.30870397e+01, 3.52514711e+00, -2.50833251e+00], [ 1.43266047e-01, 3.89623072e+00, -6.35925774e+00, 4.82766147e+00], [ 5.21043508e+00, -1.24087848e+01, 4.03867039e+00, 1.66611256e+00], [-1.62616964e+00, 4.49523187e+00, 1.49293827e+00, -4.82906803e+00], [-5.32550035e+00, 6.54626904e+00, 1.80713979e+00, -3.71524346e+00], [ 8.52821648e+00, 5.04066625e+00, -1.40587989e+01, 2.04102969e+00], [ 2.93595281e+00, -3.06397346e-01, 3.14714446e-01, -2.10323975e+00], [ 4.98420615e-01, 1.78170914e+00, -4.12535639e+00, 4.25450411e+00], [ 7.38997142e+00, -5.72321073e+00, 7.45262600e-01, 1.34930069e+00], [-4.15440385e+00, 6.22867062e+00, -1.01230961e+01, 6.17843035e+00], [ 2.07792528e+00, -5.72136214e+00, 4.17771213e+00, -3.55140612e+00], [-3.50894955e-01, -9.06159637e+00, 8.25573872e+00, 1.30818915e+00], [ 2.08076027e+00, -6.29044255e+00, -3.72162478e+00, 5.59751915e+00], [ 6.41649987e+00, -7.81817999e+00, 6.37709000e+00, -2.52895091e+00], [-1.55162711e+00, -1.08251025e+00, 2.97155097e+00, 2.23137412e+00], [ 2.10190966e+00, -4.91980606e+00, -2.27178675e+00, 5.22005273e+00], [ 1.18414034e+00, 2.73005114e+00, -5.83750096e+00, 9.72626663e-01], [ 3.22651920e+00, 6.86740954e+00, -1.71826157e+01, 5.84908904e+00], [-5.06045701e+00, 3.62704119e+00, -4.91593655e-01, 3.17341621e+00], [-2.74419043e+00, -4.92905808e+00, 8.77975503e+00, 1.19943251e+00], [ 1.68910260e+00, -7.62881275e-01, -3.81825121e+00, 1.66544122e+00], [-1.41850912e+00, 3.01157827e-01, 6.16885196e+00, -6.43409910e-01], [ 4.75118024e-01, -9.87176347e-01, -5.18221601e+00, 5.67993814e+00], [ 2.12781877e+00, -9.47540979e+00, 6.01901108e+00, 1.14987261e+00], [ 1.53029208e+00, -2.97399933e+00, -2.84992590e+00, 4.52600751e+00], [-2.09622892e+00, 3.06778827e+00, -3.00679508e-01, -8.99086910e-01], [ 1.02979002e-01, 2.91304123e+00, -1.53378332e+00, -1.15256420e-01], [-3.69967762e+00, -1.87046051e+00, 5.57814190e+00, -4.64575427e-01], [ 6.07652802e+00, -3.63843715e-01, -1.46402675e+00, -4.20425143e+00], [-4.46754718e+00, 2.72070054e+00, -5.75886394e-01, 2.77871010e-01], [ 4.30168701e+00, -3.99234273e+00, 6.21472957e+00, -3.23471929e+00], [-1.33547676e-01, -1.55900603e+00, -9.03108608e-02, 1.24187254e+00], [-1.23372550e+00, -4.13081673e-01, 3.21876899e-01, 1.04678676e+00], [-4.23059944e-03, -1.09061677e+00, -3.00313106e+00, 8.26518460e-01], [-6.77531892e-01, 8.10367423e+00, -5.39599858e-01, -2.29086400e+00], [-4.04985782e-01, -1.48474228e-01, 4.46426851e+00, -4.77073466e+00], [-2.20549139e+00, 2.81720318e-01, 1.45490609e+00, 3.90184195e-01], [ 3.75221236e+00, -2.65044039e+00, 6.12142700e-01, -3.60463600e+00], [-3.91022892e+00, 5.07079621e+00, 1.16447266e+00, -8.35833910e-01], [-2.00678462e+00, -8.14477811e-01, 3.32054504e+00, 4.04494022e-01], [ 2.73076413e+00, 2.50243358e-02, -6.69343919e+00, 5.25490363e+00], [-6.70032392e-01, -9.79653659e-01, -3.13383443e+00, 3.23171894e+00], [ 1.00355693e+01, -3.15428897e-01, -6.85498944e+00, -1.71965315e+00], [-7.46787311e-01, 3.21398901e+00, -3.22515547e+00, -1.50354665e+00], [ 6.02038065e+00, -1.65446169e+00, -4.27657528e+00, -2.97641891e+00], [-1.52994114e-01, 1.54591978e+00, 1.73308712e+00, -2.31506891e+00], [-4.89900973e+00, 4.23128824e+00, -2.41845460e+00, -1.20629559e+00], [ 2.64673111e+00, 7.71539655e-01, -6.55194620e+00, -1.21088454e+00], [-2.27117902e+00, -7.70630046e-02, 4.32084369e+00, -2.44675686e+00], [ 2.10497413e+00, -7.56822228e+00, 1.03777464e+00, 2.11920405e+00], [ 3.42850798e+00, -2.00527573e+00, -2.85043281e+00, 2.60588063e+00], [-1.88339330e+00, 7.25115328e+00, -1.72561818e+00, -3.97307221e+00], [ 2.67386571e-01, -2.26550129e+00, 2.16508534e+00, -9.46408739e-01], [ 1.89350239e-01, -4.66766504e+00, -1.49265571e+00, 4.08805268e+00], [ 2.27735891e+00, -3.63565761e+00, 4.83706578e-01, 3.01025215e+00], [-8.28497939e-01, -2.01153255e+00, 8.30862648e+00, -5.22162332e+00], [ 6.04579659e+00, -3.08949296e+00, -7.33446649e+00, 3.12003199e+00], [ 3.78526509e+00, 4.65662720e+00, -6.99794246e+00, -2.40175340e+00], [ 4.99251369e+00, -8.30421459e-01, -3.32832737e+00, -2.85243608e+00], [-3.32437284e+00, -1.13524143e+01, 9.18645386e+00, 4.32262240e+00], [-1.25159706e+00, -2.83695323e-01, 3.33656762e+00, 1.58861238e-01], [-3.49945828e+00, -2.89292527e-01, 4.71130392e+00, 2.70824258e-01], [-4.94879325e+00, 1.05565813e+01, -4.93085787e+00, 1.14245656e+00], [ 2.24091119e+00, 4.11198903e+00, -4.84521562e+00, -2.68630230e+00], [ 1.34376611e+00, 1.59324806e+00, -2.39524243e+00, 1.40507817e+00], [ 3.32239130e+00, -2.86730451e+00, 4.75682443e-01, -3.47046689e-01], [ 2.30759868e+00, 1.97606270e+00, -3.86960437e+00, -7.37651007e-01], [-2.17681697e+00, -2.68333247e+00, 6.93878178e+00, -1.41270485e+00], [ 2.99544432e+00, -2.97141770e+00, 8.40179631e-02, -1.46984216e+00], [-6.20290670e+00, -7.61475626e-02, 2.34485537e+00, 1.97685570e+00]]), np.array([ 1.48655729, -2.18673978, 0.90309447, -0.90890397]), [1.16782678467265, 0.4633313331209067, 0.3825014498861504, 0.34357854494869644, 0.31959671495104375, 0.30287611450102964, 0.2903330708139067, 0.2804625242874614, 0.27242926271577445, 0.2657257383993019, 0.26002217840170727, 0.2550931640451539, 0.25077840720579364, 0.24696029296616737, 0.24355027972207458, 0.24048027633953797, 0.23769696360289197, 0.23515793424931394, 0.23282899795583753, 0.230682256058964, 0.22869469857726063, 0.22684716390752546, 0.22512355548289478, 0.22351024376543754, 0.22199560404829685, 0.22056965520226648, 0.21922377442252938, 0.21795046986728825, 0.21674319786772406, 0.21559621479180457, 0.2145044560961892, 0.21346343688874272, 0.21246916964339735, 0.21151809569254118, 0.2106070278623564, 0.20973310217874508, 0.20889373700205768, 0.2080865982811923, 0.207309569876049, 0.20656072809964612, 0.20583831979064762, 0.2051407433534777, 0.2044665323040455, 0.20381434093999345, 0.20318293181961983, 0.2025711647865071, 0.2019779873199699, 0.20140242602669586, 0.2008435791179468, 0.20030060974062958, 0.1997727400504027, 0.19925924593151487, 0.19875945228189193, 0.1982727287935786, 0.19779848616840343, 0.19733617271697906, 0.19688527129613845, 0.19644529654585077, 0.1960157923917265, 0.1955963297835574, 0.19518650464405413, 0.19478593600514163, 0.19439426431193155, 0.19401114987687174, 0.19363627146864365, 0.19326932502216668, 0.19291002245763686, 0.19255809059788673, 0.19221327017454626, 0.19187531491452905, 0.19154399069928532, 0.19121907479006858, 0.1909003551131739, 0.19058762959973422, 0.1902807055752139, 0.1899793991942318, 0.18968353491678155, 0.18939294502230497, 0.1891074691584184, 0.1888269539214, 0.18855125246582222, 0.18828022414095477, 0.18801373415178887, 0.18775165324272627, 0.18749385740215505, 0.18724022758629427, 0.18699064946083013, 0.1867450131589984, 0.1865032130548815, 0.18626514755079532, 0.1860307188777362, 0.18579983290794297, 0.1855723989787086, 0.18534832972664672, 0.18512754093168088, 0.1849099513700854, 0.18469548267595787, 0.18448405921055375, 0.1842756079389546, 0.18407005831358694], [1.1805509350744763, 0.4689177762322239, 0.38841717278982324, 0.3504170596968216, 0.32778339733058715, 0.3125942528011321, 0.30161625496407035, 0.29326642404847214, 0.2866733965591491, 0.2813159605346815, 0.276862293364221, 0.2730906313482291, 0.2698468863912062, 0.26702052933465587, 0.26453014472895753, 0.26231440163798836, 0.2603262028048069, 0.258528774590643, 0.25689298318159226, 0.25539544898839583, 0.25401719434477676, 0.25274265584743005, 0.251558951200626, 0.25045532699638723, 0.2494227372779521, 0.24845351806411545, 0.24754113325046143, 0.2466799742667702, 0.245865200684692, 0.24509261234975963, 0.24435854601770504, 0.2436597912096207, 0.24299352126599005, 0.24235723651309898, 0.24174871715105667, 0.24116598399622838, 0.24060726560849727, 0.24007097063836474, 0.23955566446412188, 0.23906004937239114, 0.2385829476788409, 0.23812328729914575, 0.23768008937023838, 0.23725245759380306, 0.2368395690317478, 0.23644066613007592, 0.23605504978547714, 0.23568207329987098, 0.235321137093448, 0.23497168406757557, 0.23463319552611503, 0.23430518757793195, 0.23398720795520575, 0.23367883319200658, 0.23337966611585245, 0.2330893336118718, 0.232807484625011, 0.2325337883706274, 0.23226793272795207, 0.23200962279441562, 0.2317585795818159, 0.23151453883784384, 0.23127724997864976, 0.2310464751199921, 0.23082198819609473, 0.23060357415670746, 0.2303910282340419, 0.2301841552722688, 0.22998276911314144, 0.2297866920320705, 0.22959575421963743, 0.2294097933041079, 0.22922865391100888, 0.2290521872562699, 0.22888025076981441, 0.2287127077468212, 0.22854942702417344, 0.22839028267987188, 0.228235153753418, 0.22808392398537639, 0.22793648157450433, 0.22779271895099584, 0.22765253256452878, 0.22751582268592827, 0.2273824932213729, 0.2272524515381708, 0.227125608301217, 0.22700187731933216, 0.22688117540074446, 0.22676342221704945, 0.22664854017503477, 0.22653645429581212, 0.2264270921007429, 0.22632038350369071, 0.22621626070916748, 0.22611465811597842, 0.22601551222600083, 0.22591876155776108, 0.22582434656450012, 0.2257322095564404])
# (W,b,ceList,tceList) = (W1,b1,ceList1,tceList1)

In [ ]:
%matplotlib widget

In [ ]:
plt.plot([*range(len(ceList))],ceList)
plt.plot([*range(len(tceList))],tceList)
plt.show()

In [ ]:
gamma = lambda text : np.argmax(softmaxV(np.dot(docVector(text, X_normalized_100d, word2ind),W)+b))

In [ ]:
testClassifier([testEcoCorpus,testMilCorpus,testPolCorpus,testCulCorpus], gamma)